In [1]:
import pandas as pd



In [2]:
data = pd.read_csv('data/decaydata.csv')
data.dropna(inplace=True)
data['30 min decay'] = data["T0"]-data["T0.5"]
data["SUVA"] = data["UVA"]/data["FOC"]*100

In [3]:
cleaneddata = data[data["Code"] != "SW23"]

In [4]:
cleaneddata["classification"] = ["H" if x >= 250 else "M" if x >= 150 else "L" for x in cleaneddata["TTHM"]]

<ipython-input-4-0e0d718d9402>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaneddata["classification"] = ["H" if x >= 250 else "M" if x >= 150 else "L" for x in cleaneddata["TTHM"]]


In [6]:

target_names = cleaneddata['classification'].unique()[0:3]
target_names

array(['L', 'M', 'H'], dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder

y  = cleaneddata["classification"]

label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y

array([1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 1, 1, 2,
       2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1,
       1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1,
       1, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 1, 1,
       2, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2,
       2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,

In [10]:
feature_names = ["UVA","FOC","pH","Dose Rate (mg/L)","30 min decay","Br","COLOUR","Turb","Cl","COND"]

X = cleaneddata[feature_names]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=42)


In [13]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7722772277227723

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8217821782178217

In [16]:
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8217821782178217


In [15]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True) 

[(0.3070286469482181, 'Dose Rate (mg/L)'),
 (0.1303926493452933, 'FOC'),
 (0.12002327245298605, '30 min decay'),
 (0.08549363146256972, 'UVA'),
 (0.07116608147853914, 'pH'),
 (0.06899938286756428, 'Br'),
 (0.06311405811402096, 'Cl'),
 (0.05761211356553396, 'Turb'),
 (0.05666173206151821, 'COND'),
 (0.03950843170375639, 'COLOUR')]